In [1]:
import streamlit as st
import sqlite3
import json
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import altair as alt
from datetime import datetime, timedelta
import random
import re
import nltk
import emoji
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from collections import Counter

# Download NLTK resources if not already downloaded
try:
    nltk.data.find('vader_lexicon')
    nltk.data.find('punkt')
    nltk.data.find('stopwords')
except LookupError:
    nltk.download('vader_lexicon')
    nltk.download('punkt')
    nltk.download('stopwords')

# Import models and utilities from models.py
from models import DatabaseManager, EmotionAnalyzer, TextProcessor, MoodPredictor

# Set page configuration
st.set_page_config(
    page_title="MoodMate Pro",
    page_icon="😊",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Initialize database
db_manager = DatabaseManager("moodmate.db")
db_manager.setup_database()

# Initialize models
text_processor = TextProcessor()
emotion_analyzer = EmotionAnalyzer()
mood_predictor = MoodPredictor()

# App title and description
st.title("MoodMate Pro")
st.markdown("""
    A cutting-edge mood analysis and emotion tracking system that helps you understand your emotional patterns.
""")

# Create tabs
tabs = st.tabs([
    "Real-time Emotion Analysis", 
    "Mood Evolution Timeline", 
    "Predictive Insights", 
    "Clinical Settings"
])

# Function to display emotion radar chart
def display_emotion_radar(emotion_scores):
    """
    Display a radar chart of emotion scores
    """
    # Extract the main emotion dimensions
    emotions = ["Valence", "Arousal", "Dominance", "Openness", 
               "Conscientiousness", "Extraversion", "Agreeableness", "Neuroticism"]
    
    # Get scores and confidence intervals
    scores = [emotion_scores[e] for e in emotions]
    lower_bounds = [emotion_scores.get(f"{e}_lower", emotion_scores[e] - 0.1) for e in emotions]
    upper_bounds = [emotion_scores.get(f"{e}_upper", emotion_scores[e] + 0.1) for e in emotions]
    
    # Create emoji mapping for each emotion
    emoji_map = {
        "Valence": "😊",
        "Arousal": "⚡",
        "Dominance": "💪",
        "Openness": "🌟",
        "Conscientiousness": "📝",
        "Extraversion": "🎭",
        "Agreeableness": "🤝",
        "Neuroticism": "😰"
    }
    
    # Create labels with emojis
    labels = [f"{e} {emoji_map[e]}" for e in emotions]
    
    # Close the loop for the radar chart
    scores.append(scores[0])
    lower_bounds.append(lower_bounds[0])
    upper_bounds.append(upper_bounds[0])
    labels.append(labels[0])
    emotions_closed = emotions + [emotions[0]]
    
    # Create the radar chart
    fig = go.Figure()
    
    # Add confidence interval area
    fig.add_trace(go.Scatterpolar(
        r=upper_bounds,
        theta=labels,
        fill=None,
        mode='lines',
        line_color='rgba(32, 128, 255, 0.2)',
        showlegend=False
    ))
    
    fig.add_trace(go.Scatterpolar(
        r=lower_bounds,
        theta=labels,
        fill='tonext',  # Fill between this trace and the previous one
        mode='lines',
        line_color='rgba(32, 128, 255, 0.2)',
        name='3σ Confidence Interval'
    ))
    
    # Add main score line
    fig.add_trace(go.Scatterpolar(
        r=scores,
        theta=labels,
        mode='lines+markers',
        line_color='rgb(32, 128, 255)',
        marker=dict(size=10),
        name='Emotion Score'
    ))
    
    # Update layout
    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 1]
            )
        ),
        showlegend=True,
        title="Emotion Radar Chart",
        height=500
    )
    
    # Display the chart
    st.plotly_chart(fig, use_container_width=True)
    
    # Display a summary of the highest and lowest emotions
    st.subheader("Emotion Summary")
    
    # Get the top 2 and bottom 2 emotions
    emotion_values = [(e, emotion_scores[e]) for e in emotions]
    emotion_values.sort(key=lambda x: x[1], reverse=True)
    
    col1, col2 = st.columns(2)
    
    with col1:
        st.markdown("**Strongest Emotions:**")
        for emotion, score in emotion_values[:2]:
            st.markdown(f"- {emotion} {emoji_map[emotion]}: {score:.2f}")
    
    with col2:
        st.markdown("**Weakest Emotions:**")
        for emotion, score in emotion_values[-2:]:
            st.markdown(f"- {emotion} {emoji_map[emotion]}: {score:.2f}")

# Function to display mood timeline
def display_mood_timeline(mood_data):
    """
    Display a timeline of mood evolution
    """
    if mood_data.empty:
        st.info("No mood data available for the selected date range.")
        return
    
    # Ensure timestamp is datetime
    mood_data['timestamp'] = pd.to_datetime(mood_data['timestamp'])
    
    # Create a valence timeline
    st.subheader("Mood Valence Over Time")
    
    # Create the base chart
    valence_chart = alt.Chart(mood_data).mark_line(point=True).encode(
        x=alt.X('timestamp:T', title='Date'),
        y=alt.Y('Valence:Q', title='Valence', scale=alt.Scale(domain=[0, 1])),
        tooltip=['timestamp:T', 'Valence:Q', 'text:N']
    ).properties(
        height=300
    ).interactive()
    
    # Add a smoothed trend line using Gaussian process regression
    if len(mood_data) >= 3:  # Need at least 3 points for a meaningful trend
        # Create a dense time range for smoother curve
        min_date = mood_data['timestamp'].min()
        max_date = mood_data['timestamp'].max()
        
        if min_date != max_date:
            dense_dates = pd.date_range(min_date, max_date, periods=100)
            trend_data = pd.DataFrame({'timestamp': dense_dates})
            
            # Use Altair's loess transform for smoothing
            trend_line = alt.Chart(mood_data).transform_loess(
                'timestamp', 'Valence', bandwidth=0.5
            ).mark_line(color='red', strokeDash=[5, 5]).encode(
                x='timestamp:T',
                y='Valence:Q'
            )
            
            # Combine the charts
            final_chart = valence_chart + trend_line
        else:
            final_chart = valence_chart
    else:
        final_chart = valence_chart
    
    st.altair_chart(final_chart, use_container_width=True)
    
    # Create a multi-emotion timeline
    st.subheader("Multiple Emotions Over Time")
    
    # Select emotions to display
    emotions = ["Valence", "Arousal", "Dominance"]
    
    # Melt the dataframe to get it in the right format for Altair
    emotion_data = mood_data[['timestamp'] + emotions].melt(
        id_vars=['timestamp'],
        value_vars=emotions,
        var_name='Emotion',
        value_name='Score'
    )
    
    # Create the multi-line chart
    multi_chart = alt.Chart(emotion_data).mark_line(point=True).encode(
        x=alt.X('timestamp:T', title='Date'),
        y=alt.Y('Score:Q', title='Score', scale=alt.Scale(domain=[0, 1])),
        color=alt.Color('Emotion:N', legend=alt.Legend(title="Emotion")),
        tooltip=['timestamp:T', 'Emotion:N', 'Score:Q']
    ).properties(
        height=300
    ).interactive()
    
    st.altair_chart(multi_chart, use_container_width=True)
    
    # Display text entries
    st.subheader("Your Journal Entries")
    
    # Sort by most recent first
    sorted_entries = mood_data.sort_values('timestamp', ascending=False)
    
    for _, entry in sorted_entries.iterrows():
        with st.expander(f"{entry['timestamp'].strftime('%Y-%m-%d %H:%M')} - Valence: {entry['Valence']:.2f}"):
            st.write(entry['text'])

# Function to display predictive insights
def display_predictive_insights(mood_data):
    """
    Display predictive analytics for mood trends
    """
    if mood_data.empty or len(mood_data) < 5:  # Need at least 5 data points
        st.info("Not enough data for predictive insights. Continue adding entries.")
        return
    
    # Ensure timestamp is datetime
    mood_data['timestamp'] = pd.to_datetime(mood_data['timestamp'])
    
    # Convert timestamps to numeric (days since first entry)
    first_day = mood_data['timestamp'].min()
    mood_data['days'] = (mood_data['timestamp'] - first_day).dt.total_seconds() / (24 * 3600)
    
    # Create prediction for next 7 days
    last_day = mood_data['days'].max()
    future_days = np.linspace(last_day + 1, last_day + 7, 7)
    
    # Prepare data for polynomial regression
    X = mood_data['days'].values.reshape(-1, 1)
    
    # Create predictions for each emotion
    emotions = ["Valence", "Arousal", "Dominance"]
    
    for emotion in emotions:
        st.subheader(f"Predicted {emotion} Trend")
        
        y = mood_data[emotion].values
        
        # Create polynomial features
        poly = PolynomialFeatures(degree=2)
        X_poly = poly.fit_transform(X)
        
        # Fit polynomial regression model
        model = LinearRegression()
        model.fit(X_poly, y)
        
        # Predict future values
        future_X = np.linspace(0, last_day + 7, 100).reshape(-1, 1)
        future_X_poly = poly.transform(future_X)
        predictions = model.predict(future_X_poly)
        
        # Ensure predictions are between 0 and 1
        predictions = np.clip(predictions, 0, 1)
        
        # Create dates for the x-axis
        future_dates = [first_day + timedelta(days=float(d)) for d in future_X.flatten()]
        
        # Create the plot
        fig = go.Figure()
        
        # Add historical data
        fig.add_trace(go.Scatter(
            x=mood_data['timestamp'],
            y=mood_data[emotion],
            mode='markers',
            name='Historical Data',
            marker=dict(size=8)
        ))
        
        # Add prediction line
        fig.add_trace(go.Scatter(
            x=future_dates,
            y=predictions,
            mode='lines',
            name='Prediction',
            line=dict(color='red', dash='dash')
        ))
        
        # Add vertical line to separate historical data from predictions
        fig.add_vline(
            x=mood_data['timestamp'].max(),
            line_dash="dash",
            line_color="gray",
            annotation_text="Today",
            annotation_position="top right"
        )
        
        # Update layout
        fig.update_layout(
            xaxis_title='Date',
            yaxis_title=emotion,
            yaxis=dict(range=[0, 1]),
            height=400
        )
        
        st.plotly_chart(fig, use_container_width=True)
    
    # Mood pattern insights
    st.subheader("Mood Pattern Insights")
    
    # Check for day of week patterns
    mood_data['day_of_week'] = mood_data['timestamp'].dt.day_name()
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    
    # Group by day of week and calculate average valence
    day_valence = mood_data.groupby('day_of_week')['Valence'].mean().reindex(day_order)
    
    if not day_valence.empty and len(day_valence) > 1:
        fig = go.Figure(go.Bar(
            x=day_valence.index,
            y=day_valence.values,
            marker_color='skyblue'
        ))
        
        fig.update_layout(
            title="Average Valence by Day of Week",
            xaxis_title="Day of Week",
            yaxis_title="Average Valence",
            yaxis=dict(range=[0, 1]),
            height=400
        )
        
        st.plotly_chart(fig, use_container_width=True)
        
        # Find best and worst days
        best_day = day_valence.idxmax()
        worst_day = day_valence.idxmin()
        
        st.markdown(f"**Best day of the week:** {best_day} (Average Valence: {day_valence[best_day]:.2f})")
        st.markdown(f"**Challenging day of the week:** {worst_day} (Average Valence: {day_valence[worst_day]:.2f})")

# Function to display chatbot
def display_chatbot(user_input):
    """
    Display a simple chatbot interface that responds to user input
    with mood-related insights and suggestions
    """
    st.markdown("Chat with MoodMate Assistant for insights and suggestions.")
    
    # Initialize chat history if it doesn't exist
    if "chat_history" not in st.session_state:
        st.session_state.chat_history = [
            {"role": "assistant", "content": "Hello! I'm your MoodMate Assistant. How are you feeling today?"}
        ]

    # Display chat history
    for message in st.session_state.chat_history:
        if message["role"] == "user":
            st.markdown(f"**You:** {message['content']}")
        else:
            st.markdown(f"**MoodMate:** {message['content']}")

    # Chat input box and submit logic
    with st.form(key="chat_form", clear_on_submit=True):
        chat_input = st.text_input("Type your message:", key="chat_input")
        submit_button = st.form_submit_button("Send")

        if submit_button and chat_input:
            # Add user message to chat history
            st.session_state.chat_history.append({"role": "user", "content": chat_input})

            # Generate response based on user input
            response = generate_response(chat_input)

            # Add assistant response to chat history
            st.session_state.chat_history.append({"role": "assistant", "content": response})

def generate_response(user_input):
    """
    Generate a response based on user input
    In a production environment, this would use a more sophisticated NLP model
    """
    # Convert input to lowercase for easier matching
    user_input_lower = user_input.lower()
    
    # Check for common greetings
    if any(greeting in user_input_lower for greeting in ["hello", "hi", "hey", "greetings"]):
        return "Hello! How are you feeling today?"
    
    # Check for mood-related keywords
    if any(word in user_input_lower for word in ["sad", "unhappy", "depressed", "down", "blue"]):
        responses = [
            "I'm sorry to hear you're feeling down. Remember that emotions are temporary and this will pass.",
            "It sounds like you're having a difficult time. Consider doing something you enjoy today, even if it's small.",
            "When you're feeling sad, it can help to talk to someone you trust or practice some self-care activities."
        ]
        return random.choice(responses)
    
    if any(word in user_input_lower for word in ["happy", "good", "great", "excellent", "amazing"]):
        responses = [
            "I'm glad to hear you're feeling positive! What's contributing to your good mood?",
            "That's wonderful! Positive emotions are worth savoring and reflecting on.",
            "Great to hear! Remember to take note of what makes you feel good so you can return to it later."
        ]
        return random.choice(responses)
    
    if any(word in user_input_lower for word in ["anxious", "worried", "nervous", "stress", "stressed"]):
        responses = [
            "Anxiety can be challenging. Have you tried any breathing exercises or mindfulness techniques?",
            "When feeling anxious, it can help to focus on what's in your control and take small steps.",
            "Stress is a natural response, but it's important to find healthy ways to manage it. What usually helps you relax?"
        ]
        return random.choice(responses)
    
    # Check for questions about the app
    if "how" in user_input_lower and any(word in user_input_lower for word in ["work", "use", "track", "analyze"]):
        return "MoodMate Pro analyzes your text entries to track your emotional patterns over time. The more you share, the more insights you'll receive!"
    
    # Check for gratitude
    if any(word in user_input_lower for word in ["thanks", "thank you", "appreciate"]):
        return "You're welcome! I'm here to support your emotional well-being journey."
    
    # Default responses
    default_responses = [
        "Tell me more about how you're feeling today.",
        "I'm here to listen. Would you like to share more about your emotions?",
        "Your emotional well-being matters. Is there anything specific you'd like to discuss?",
        "Remember that tracking your moods regularly can help you understand your emotional patterns better.",
        f"It's {datetime.now().strftime('%A')}. How has your day been so far?"
    ]
    
    return random.choice(default_responses)

# Tab 1: Real-time Emotion Analysis
with tabs[0]:
    st.header("Real-time Emotion Analysis")
    
    # Create a responsive layout using golden ratio (approximately 1.618)
    col1, col2 = st.columns([1, 1.618])
    
    with col1:
        st.subheader("Express Your Thoughts")
        user_input = st.text_area("What's on your mind today?", height=200)
        
        if st.button("Analyze"):
            if user_input:
                # Process text and analyze emotions
                processed_text = text_processor.preprocess(user_input)
                emotion_scores = emotion_analyzer.analyze(processed_text)
                
                # Save to database
                timestamp = datetime.now()
                db_manager.save_entry(user_input, emotion_scores, timestamp)
                
                st.success("Your entry has been analyzed and saved!")
            else:
                st.warning("Please enter some text to analyze.")
    
    with col2:
        if 'emotion_scores' in locals():
            display_emotion_radar(emotion_scores)
        else:
            # Get the most recent entry if available
            recent_entry = db_manager.get_latest_entry()
            if recent_entry:
                display_emotion_radar(recent_entry['emotion_scores'])
            else:
                st.info("Enter your thoughts to see your emotional analysis here.")
        
        # Display chatbot interface
        st.subheader("MoodMate Assistant")
        display_chatbot(user_input if 'user_input' in locals() and user_input else "")

# Tab 2: Mood Evolution Timeline
with tabs[1]:
    st.header("Mood Evolution Timeline")
    
    # Date range selector
    col1, col2 = st.columns(2)
    with col1:
        start_date = st.date_input("Start Date", datetime.now() - timedelta(days=30))
    with col2:
        end_date = st.date_input("End Date", datetime.now())
    
    # Get mood data from database
    mood_data = db_manager.get_mood_data(start_date, end_date)
    
    if mood_data.empty:
        st.info("No mood data available for the selected date range. Start adding entries to see your mood evolution.")
    else:
        display_mood_timeline(mood_data)

# Tab 3: Predictive Insights
with tabs[2]:
    st.header("Predictive Insights")
    
    if db_manager.has_sufficient_data():
        mood_data = db_manager.get_all_mood_data()
        display_predictive_insights(mood_data)
    else:
        st.info("Not enough data for predictive insights. Continue adding entries to unlock this feature.")

# Tab 4: Clinical Settings
with tabs[3]:
    st.header("Clinical Settings")
    
    st.subheader("Analysis Parameters")
    
    col1, col2 = st.columns(2)
    
    with col1:
        st.slider("Sensitivity Threshold", 0.0, 1.0, 0.5, 0.01)
        st.selectbox("Primary Emotion Model", ["BERT-based", "RoBERTa", "DistilBERT"])
    
    with col2:
        st.slider("Confidence Interval (σ)", 1.0, 3.0, 2.0, 0.1)
        st.selectbox("Secondary Analysis", ["Big Five Personality", "OCEAN Model", "None"])
    
    st.subheader("Data Management")
    
    col1, col2 = st.columns(2)
    
    with col1:
        if st.button("Export Data"):
            data = db_manager.get_all_mood_data()
            if not data.empty:
                csv = data.to_csv(index=False)
                st.download_button(
                    label="Download CSV",
                    data=csv,
                    file_name="moodmate_data.csv",
                    mime="text/csv"
                )
            else:
                st.warning("No data to export.")
    
    with col2:
        if st.button("Clear All Data"):
            st.warning("This will permanently delete all your mood data.")
            if st.button("Confirm Delete"):
                db_manager.clear_all_data()
                st.success("All data has been cleared.")

# Sidebar for user profile and settings
with st.sidebar:
    st.header("User Profile")
    
    # Placeholder for user authentication
    if not st.session_state.get("authenticated", False):
        st.session_state.authenticated = True
        st.session_state.username = "Guest User"
    
    st.write(f"Welcome, {st.session_state.username}!")
    
    st.subheader("Calendar Integration")
    calendar_option = st.selectbox("Select Calendar", ["None", "Google Calendar", "Microsoft Outlook"])
    
    if calendar_option != "None":
        st.info(f"{calendar_option} integration is available in the premium version.")
    
    st.subheader("Theme")
    theme = st.selectbox("Select Theme", ["Light", "Dark", "System Default"])
    
    st.subheader("About")
    st.markdown("""
        **MoodMate Pro** v1.0.0  
        A cutting-edge mood analysis and emotion tracking system.
    """)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\kamal\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kamal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kamal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2025-04-13 12:13:25.808 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kamal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kamal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downl